<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/Image/05_conditional_operations.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/Image/05_conditional_operations.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/Image/05_conditional_operations.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Relational, conditional and Boolean operations
To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with `And()`:

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Add Earth Engine Python script 

In [ ]:
# Load a Landsat 8 image.
image = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318")

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(["B5", "B4"])
ndwi = image.normalizedDifference(["B3", "B5"])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
Map.addLayer(bare.updateMask(bare), {}, "bare")

Map.addLayerControl()
Map

As illustrated by this example, the output of relational and boolean operators is either True (1) or False (0). To mask the 0's, you can mask the resultant binary image with itself. 

The binary images that are returned by relational and boolean operators can be used with mathematical operators. This example creates zones of urbanization in a nighttime lights image using relational operators and `image.add()`:

In [ ]:
Map = geemap.Map()

# Load a 2012 nightlights image.
nl2012 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012")
lights = nl2012.select("stable_lights")
Map.addLayer(lights, {}, "Nighttime lights")

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Display the thresholded image as three distinct zones near Paris.
palette = ["000000", "0000FF", "00FF00", "FF0000"]
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {"min": 0, "max": 3, "palette": palette}, "development zones")

Map.addLayerControl()
Map

Note that the code in the previous example is equivalent to using a [ternary operator](http://en.wikipedia.org/wiki/%3F:) implemented by `expression()`:

In [ ]:
Map = geemap.Map()

# Create zones using an expression, display.
zonesExp = nl2012.expression(
    "(b('stable_lights') > 62) ? 3"
    + ": (b('stable_lights') > 55) ? 2"
    + ": (b('stable_lights') > 30) ? 1"
    + ": 0"
)
Map.addLayer(
    zonesExp, {"min": 0, "max": 3, "palette": palette}, "development zones (ternary)"
)
Map.setCenter(2.373, 48.8683, 8)

Map.addLayerControl()
Map

Observe that in the previous expression example, the band of interest is referenced using the`b()` function, rather than a dictionary of variable names. (Learn more about image expressions on [this page](https://developers.google.com/earth-engine/image_math#expressions). Using either mathematical operators or an expression, the output is the same and should look something like Figure 2.

Another way to implement conditional operations on images is with the `image.where()` operator. Consider the need to replace masked pixels with some other data. In the following example, cloudy pixels are replaced by pixels from a cloud-free image using `where()`:

In [ ]:
Map = geemap.Map()

# Load a cloudy Landsat 8 image.
image = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603")
Map.addLayer(
    image, {"bands": ["B5", "B4", "B3"], "min": 0, "max": 0.5}, "original image"
)

# Load another image to replace the cloudy pixels.
replacement = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416")

# Compute a cloud score band.
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select("cloud")

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display the result.
Map.centerObject(image, 9)
Map.addLayer(
    replaced, {"bands": ["B5", "B4", "B3"], "min": 0, "max": 0.5}, "clouds replaced"
)

Map.addLayerControl()
Map

In this example, observe the use of the `simpleCloudScore()` algorithm. This algorithm ranks pixels by cloudiness on a scale of 0-100, with 100 most cloudy. Learn more about `simpleCloudScore()` on the [Landsat Algorithms page](https://developers.google.com/earth-engine/landsat#simple-cloud-score).